In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install dipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly_express as px
# px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

In [ ]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
wayid_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

# map support point with lat lon

In [ ]:
dfs = []
day_len = []
day_trip_count = []
# days = range(1,32)
days = range(1,2)
# months = range(1,4)
months = range(1,2)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(wayid_data_path + f'/trips_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df.sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

[195862]
[2099]


,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,month,day
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0,340294191,0.004308,1,1
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0,92193150,0.000498,1,1
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0,402348153,0.004740,1,1
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0,92193150,0.008590,1,1
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0,92193150,0.002386,1,1
...,...,...,...,...,...,...,...,...,...,...,...
195857,2019-01-01 21:51:58,003000300000359769033400604,14.835272,101.550918,52,7.0,2098,82374801,0.001614,1,1
195858,2019-01-01 21:53:56,003000300000359769033400604,14.846673,101.562518,61,7.0,2098,82374801,0.001829,1,1
195859,2019-01-01 21:56:00,003000300000359769033400604,14.850647,101.572147,59,7.0,2098,82374801,0.007574,1,1
195860,2019-01-01 21:57:55,003000300000359769033400604,14.844158,101.590907,60,7.0,2098,82374801,0.005860,1,1


In [ ]:
way_seq = df.way_id.to_list()
not_stop_mask = [True]+[(a != b) for a, b in zip(way_seq[:-1], way_seq[1:])]
df = df.loc[not_stop_mask]

In [ ]:
way_df = df[['way_id', 'lat', 'lon']]
way_df = way_df.drop_duplicates(subset='way_id')
way_df

,way_id,lat,lon
0,340294191,16.481607,104.049705
1,92193150,16.491331,104.052101
2,402348153,16.502571,104.054756
5,402340885,16.533987,104.055549
10,820191737,16.561748,104.021415
...,...,...,...
194789,40268077,17.103240,101.172262
194791,541463213,17.185505,101.159089
194792,40268079,17.224956,101.190091
195264,668085796,14.702768,102.087105


In [ ]:
loc_df = df[['trip_id', 'way_id', 'month','day']].merge(way_df,how='left')
print(sp_seq_df.isna().any())
loc_df

time_stamp    False
unit_id       False
lat           False
lon           False
speed         False
unit_type     False
trip_id       False
way_id        False
dist          False
month         False
day           False
dtype: bool


,trip_id,way_id,month,day,lat,lon
0,0,340294191,1,1,16.481607,104.049705
1,0,92193150,1,1,16.491331,104.052101
2,0,402348153,1,1,16.502571,104.054756
3,0,92193150,1,1,16.491331,104.052101
4,0,402340885,1,1,16.533987,104.055549
...,...,...,...,...,...,...
54358,2098,783299711,1,1,14.717873,101.493665
54359,2098,150586966,1,1,14.734838,101.508443
54360,2098,783299693,1,1,14.772588,101.520985
54361,2098,82374801,1,1,14.756580,101.516680


partition trips

In [ ]:
trips = []
low_12 = 0
for month in months:
  for day in days:
    day_df = loc_df[(loc_df.day == day) & (loc_df.month == month)]
    for t_id in day_df.trip_id.unique():
      trip = day_df[day_df.trip_id == t_id][['lat', 'lon']]
      if len(trip)/12 > 1.5:
        split_trip=[trip.iloc[i:i + 24] for i in range(0, len(trip), 12)]
        if len(split_trip[-1]) < 16:
          last = split_trip.pop()
          before_last = split_trip.pop()
          merge_last = pd.concat([before_last, last])
          split_trip.append(merge_last)
        for partial_trip in split_trip:
          lat_lon_data = np.c_[partial_trip['lat'].values, partial_trip['lon'].values]
          trips.append(lat_lon_data)
      else:
        # print(len(trip))
        lat_lon_data = np.c_[trip['lat'].values, trip['lon'].values]
        trips.append(lat_lon_data)
# print(low_12)
print(len(trips))
trips[0][0,0], trips[0][0,1]

3887


(16.481607, 104.049705)

In [ ]:
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.2'

In [ ]:
import pickle
import collections
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import VectorOfEndpointsFeature
from dipy.segment.clustering import QuickBundles
class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=12))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
        currD = np.mean(x)
        return currD

THRESHOLD = 24
metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD, metric=metric)

# feature = ResampleFeature(nb_points=24)
# metric = AveragePointwiseEuclideanMetric(feature=feature)
# qb = QuickBundles(threshold=THRESHOLD, metric=metric)

clusters = qb.cluster(trips)

pkl_filename = train_path+"/QBcluster_test_way_treshold24.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clusters, file)
with open(pkl_filename, 'rb') as file:
    clusters = pickle.load(file)

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", collections.Counter(clusters < 10)[True], end=' /')
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

Nb. clusters: 547
Cluster sizes: <map object at 0x7f5494547990>
Small clusters: 439 /Small clusters: [ True False False False False False False False  True False False False
 False  True  True False False False False  True False False False False
 False False False False False False False False False False False False
  True False False False False False False False  True False  True False
 False False False  True False False False False False False False  True
 False  True  True False False False  True False False  True  True False
  True False False False False  True  True  True False False False False
 False False False False  True False False False  True  True False  True
  True  True  True False False  True  True  True False False  True  True
 False  True False  True  True  True False  True  True False  True  True
 False False False  True  True  True False  True  True  True False  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  Tru

In [ ]:
loc_df['route_cluster'] = pd.Series()

cluster_list = []
index = 0
for trip in trips:
  lenght = len(trip)
  for cluster in range(len(clusters)):
      if index in clusters[cluster].indices:
        cluster_list.extend([cluster]*lenght)
  index+=1
print(len(cluster_list))
loc_df.route_cluster = cluster_list
loc_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


83817


ValueError: ignored

In [ ]:
df[['day', 'month', 'route_cluster']] = loc_df[['day', 'month', 'route_cluster']]
for month in months:
  for day in days:
    day_df = df[(df.day == day) & (df.month == month)]
    day_df = day_df.drop(['day'], axis=1)
    day_df = day_df.drop(['month'], axis=1)
    day_df.to_csv(wayid_data_path + f'/lebeled/testtrips_2019_{month}_{day}.csv', encoding='utf-8', index=False)
    print(day_df.route_cluster.unique())
    print(day_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


[0.000e+00 1.000e+00 9.000e+00 1.000e+01 1.100e+01 1.200e+01 1.500e+01
 1.600e+01 1.700e+01 1.800e+01 2.000e+01 2.100e+01 2.200e+01 2.300e+01
 2.600e+01 2.700e+01 2.800e+01 2.900e+01 3.100e+01 3.200e+01 3.300e+01
 3.400e+01 3.500e+01 3.600e+01 3.700e+01 3.800e+01 4.300e+01 4.400e+01
 4.500e+01 4.600e+01 4.700e+01 4.800e+01 4.900e+01 5.000e+01 5.100e+01
 5.300e+01 2.400e+01 5.400e+01 5.500e+01 5.600e+01 5.700e+01 1.900e+01
 5.800e+01 2.500e+01 6.100e+01 3.000e+00 6.200e+01 6.500e+01 6.600e+01
 6.000e+01 6.800e+01 6.900e+01 7.000e+01 7.100e+01 7.200e+01 7.300e+01
 7.400e+01 7.500e+01 7.600e+01 7.700e+01 7.800e+01 8.000e+01 8.100e+01
 4.100e+01 4.200e+01 8.300e+01 8.400e+01 8.200e+01 4.000e+01 6.400e+01
 1.300e+01 7.900e+01 8.600e+01 5.900e+01 6.700e+01 5.200e+01 8.800e+01
 8.900e+01 9.000e+01 9.100e+01 9.200e+01 8.500e+01 9.300e+01 9.400e+01
 9.500e+01 9.600e+01 9.700e+01 9.800e+01 9.900e+01 1.020e+02 1.030e+02
 1.040e+02 1.050e+02 1.060e+02 1.400e+01 1.070e+02 1.090e+02 1.100e+02
 8.700